## GFS ENSEMBLE FORECAST SYSTEM (GEFS) Geopotential Height

In [1]:
import boto3
from botocore import UNSIGNED
from botocore.config import Config

from datetime import datetime, timedelta
from netCDF4 import Dataset

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import glob

import numpy as np
import os

In [68]:
dt = datetime(2023,12,20,6,0)

n_members = 30

gefs_bucket = 'noaa-gefs-pds'

s3 = boto3.resource('s3', config=Config(signature_version=UNSIGNED))
s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
bucket = s3.Bucket(gefs_bucket)

control_path = f'gefs.{dt:%Y}{dt:%m}{dt:%d}/{dt:%H}/atmos/pgrb2ap5/gec00.t{dt:%H}z.pgrb2a.0p50.f'

objs = bucket.objects.filter(Prefix=control_path)
files = np.array([i.key for i in objs])
control_files = np.array([i for i in files if "idx" not in i],dtype="S59")

ensemble_files = np.empty((n_members,len(control_files)),dtype="S59")

for i in range(n_members):
    member_path = f'gefs.{dt:%Y}{dt:%m}{dt:%d}/{dt:%H}/atmos/pgrb2ap5/gep{i+1:02}.t{dt:%H}z.pgrb2a.0p50.f'
    objs = bucket.objects.filter(Prefix=member_path)
    files = np.array([i.key for i in objs])
    member_files = np.array([i for i in files if "idx" not in i])
    #print(member_files)
    ensemble_files[i] = member_files


### Grab every 12 hourly file



In [77]:
fhours = np.arange(0,385,12)

all_files = np.empty((len(fhours),n_members+1),dtype="S59")


for h in range(len(fhours)):
    all_files[h,0] = [i for i in control_files if b"f%03i"%(fhours[h]) in i][0]
    for m in range(n_members):
        all_files[h,m+1] = [i for i in ensemble_files[m] if b"f%03i"%(fhours[h]) in i][0]


### Download all the files -_-

In [ ]:
for h in range(len(fhours)):
    for m in range(n_members):
        s3_client.download_file(gefs_bucket, all_files[h,m], 'gefs_m.nc')
